# Example to use the custom Container for mono reconstruction

In [6]:
from lstchain.io.containers import DL1ParametersContainer
from lstchain.reco.utils import guess_type

from ctapipe.utils import get_dataset_path
from ctapipe.io import HDF5TableWriter, HDF5TableReader
from ctapipe.calib import CameraCalibrator
from ctapipe.image import tailcuts_clean
from ctapipe.io import event_source
from ctapipe.image import hillas_parameters

## Options

In [7]:
infile = get_dataset_path('gamma_test_large.simtel.gz')   

dl1_parameters_filename = 'dl1.h5'

allowed_tels = {1} # select LST1 only
max_events = 300 # limit the number of events to analyse in files - None if no limit

cal = CameraCalibrator(r1_product='HESSIOR1Calibrator', extractor_product='NeighbourPeakIntegrator')

cleaning_method = tailcuts_clean
cleaning_parameters = {'boundary_thresh': 0,
                       'picture_thresh': 6,
                       'min_number_picture_neighbors': 2}

## R0 to DL1

In [8]:
dl1_container = DL1ParametersContainer() 

In [13]:
with HDF5TableWriter(filename=dl1_parameters_filename, group_name='events', overwrite=True) as writer:

    source = event_source(infile)
    source.allowed_tels = allowed_tels
    source.max_events = max_events

    for i, event in enumerate(source):
        if i%100==0:
            print(i)
        cal.calibrate(event)

        for telescope_id, dl1 in event.dl1.tel.items():
            tel = event.inst.subarray.tels[telescope_id]
            camera = tel.camera
            signal_pixels = cleaning_method(camera, dl1.image[0], 
                                  **cleaning_parameters)

            image = dl1.image[0]
            image[~signal_pixels] = 0
            
            peakpos = dl1.peakpos[0]

            if image.sum() > 0:
                try:
                    hillas = hillas_parameters(
                        camera,
                        image
                    )
                    
                except:
                    break

                ## Fill container ##
                dl1_container.fill_mc(event)
                dl1_container.fill_hillas(hillas)
                dl1_container.fill_event_info(event)
                dl1_container.set_mc_core_distance(event, telescope_id)
                dl1_container.mc_type = guess_type(infile)
                dl1_container.set_timing_features(camera, image, peakpos, hillas)
                dl1_container.set_source_camera_position(event, telescope_id)
                dl1_container.set_disp([dl1_container.src_x, dl1_container.src_y], hillas)


                ## Save parameters for later training ##
                writer.write(camera.cam_id, [dl1_container])
                
                

0


In [14]:
# The file has been created
!ls -lsh dl1.h5

152 -rw-r--r--  1 thomasvuillaume  staff    75K Nov 19 14:31 dl1.h5


## Transparent data reading into the container

In [15]:
from ctapipe.io import HDF5TableReader
with HDF5TableReader(dl1_parameters_filename, mode='r+') as table:
    for c in table.read('/events/LSTCam', DL1ParametersContainer()):
        print(c.disp)

0.38452644819745496 m
0.42535687853541176 m
0.11690725839005942 m
0.6982014871100171 m
0.2138212721198705 m
0.32814469306717353 m
0.6891250070371784 m
0.692541440708613 m
0.25380699776791366 m
0.2405854011183803 m
1.3089408277160428 m
0.5015959550851092 m
0.3915556546219001 m
0.466018776475729 m
0.2810018923497003 m
0.9942298613758468 m


## The hdf5 file is also very easy to read with pandas

In [16]:
import pandas as pd
pd.read_hdf(dl1_parameters_filename, key='events/LSTCam')

,disp,event_id,gps_time,intensity,intercept,kurtosis,length,mc_alt,mc_alt_tel,mc_az,...,phi,psi,r,skewness,src_x,src_y,time_gradient,width,x,y
0,0.384526,153614,57576.867777,89.371840,8.444337,2.498454,0.047015,1.203404,1.22173,0.046534,...,2.063250,1.244666,0.964152,-0.179795,-0.503035,0.467968,-6.322201e+00,0.024291,-0.455841,0.849587
1,0.425357,192801,57576.877092,162.968549,9.181818,2.504447,0.068518,1.174510,1.22173,6.277897,...,2.904810,-2.190425,1.055906,0.221488,-1.323027,-0.057215,-9.489777e+00,0.037776,-1.026444,0.247690
2,0.116907,222202,57576.877237,202.425010,8.990935,2.846305,0.057455,1.252226,1.22173,6.271463,...,-0.100999,0.291934,0.744952,-0.314901,0.854726,-0.102846,4.338191e+00,0.036882,0.741156,-0.075112
3,0.698201,396207,57576.878666,111.718404,9.215516,2.014384,0.064035,1.236999,1.22173,6.234098,...,-1.602052,-2.053223,0.968607,-0.213353,0.437997,-0.450242,1.750502e+01,0.042401,-0.030269,-0.968134
4,0.213821,650211,57576.879971,102.341434,9.108413,2.553043,0.057854,1.199205,1.22173,6.179521,...,-2.249792,-1.090236,1.143543,-0.818150,-0.579898,-1.053016,-3.845146e+00,0.013781,-0.718159,-0.889910
5,0.328145,856512,57576.870201,116.701921,9.028011,3.330128,0.037925,1.234354,1.22173,6.245500,...,-1.421373,-2.694406,0.521978,-0.762455,0.359682,-0.348327,-2.986112e+00,0.030092,0.077706,-0.516162
6,0.689125,869911,57576.881068,352.891266,9.991287,2.410915,0.123545,1.255599,1.22173,6.279010,...,0.682866,2.014884,0.999484,0.547818,0.948764,-0.036260,8.094717e+00,0.040169,0.775367,0.630693
7,0.692541,940800,57576.881763,594.314273,11.921446,1.922396,0.145394,1.207581,1.22173,6.135493,...,-1.657167,-1.878074,0.813911,0.274622,-0.294832,-1.465980,-1.024468e+01,0.042024,-0.070211,-0.810878
8,0.253807,952004,57576.881819,151.598627,8.890541,2.120986,0.056474,1.195556,1.22173,0.019503,...,2.550244,-2.822572,0.640407,-0.076205,-0.731228,0.200201,2.094116e+00,0.031908,-0.531659,0.357015
9,0.240585,998705,57576.870919,86.879849,8.338210,1.926093,0.049935,1.232016,1.22173,6.279502,...,-0.878424,-2.325832,0.344222,-0.705910,0.288057,-0.034277,-1.236854e+01,0.022317,0.219739,-0.264959
